In [7]:
import pandas as pd

class Operation:
    """Read or Write"""
    def __init__(self, _id, target, function):
        self.id = _id
        self.target = target
        self.function = function
        self.has_lock = False
        self.has_applied = False

    def apply(self) -> bool :
        success = self.target.apply(self)
        self.has_applied = success
        return success

    def get_lock(self):
        success = self.target.lock(self)
        self.has_lock = success
        return success
    
    def free_lock(self):
        success = self.target.unlock(self)
        self.has_lock = not success
        return success



In [12]:
class Transaction:
    def __init__(self, _id, operation_list):
        self.id = _id
        self.operation_list = operation_list
        self.operation_iter = 0
        self.finished = False

    def apply_next(self) -> bool:
        if self.finished:
            return True # Transaction has finished, skip
        # Otherwise, run transaction:
        current_operation = self.operation_list[self.operation_iter]
        if current_operation.has_lock:
            if current_operation.has_applied:
                print(self.id, "Free lock of current operation", current_operation.id)
                success = current_operation.free_lock()
            else:
                print(self.id, 'Apply current operation', current_operation.id)
                current_operation.apply()
                success = False
        else:
            print(self.id, 'Get lock of current operation', current_operation.id)
            current_operation.get_lock()
            success = False
        if success:
            self.operation_iter += 1 
            if self.operation_iter==len(self.operation_list):
                self.finished = True
        return success

In [14]:
class Target:
    def __init__(self, df: pd.DataFrame):
        self.df = df # DataFrame 
        self.locked = False
        self.op_with_lock = None

    def apply(self, op: Operation) -> bool:
        if self.locked and self.op_with_lock==op:
            try:
                self.df = op.function(self.df)
                return True
            except Exception as e:
                raise e # Do something. If an operations fails to many times, it won't ensure Atomicity
                return False
        else:
            return False

    def lock(self, op: Operation) -> bool:
        if not self.locked and self.op_with_lock==None:
            self.locked = True
            self.op_with_lock = op
            return True # Success
        else:
            return False

    def unlock(self, op: Operation) -> bool:
        if self.locked and self.op_with_lock==op:
            self.locked = False
            self.op_with_lock = None
            return True
        else:
            return False


In [33]:
from random import randint
from copy import copy
from typing import Callable, Iterator, Union, Optional, List

def run_transactions(max_iterations=10, *transactions: List[Transaction]):
    non_finished_transa = copy(list(transactions))
    curr_iteration = 0
    while len(non_finished_transa) > 0 and curr_iteration < max_iterations: 
        curr_iteration += 1
        n_transa = len(non_finished_transa)
        # Simulate random access to memory : we try to run the next step of a random transaction
        i = randint(0, n_transa-1) 
        curr_transa = non_finished_transa[i]
        curr_transa.apply_next()
        # Remove transactions from list if finished
        if curr_transa.finished:
            non_finished_transa.pop(i)



In [62]:
df = pd.DataFrame({"test": [1,2,3,55]})
test_target = Target(df)

def append_value(df: pd.DataFrame, value) -> pd.DataFrame:
    max_index = max(df.index) 
    df.loc[max_index + 1] = value 
    return df 

test_function = lambda df: append_value(df, 5)

test_transa_1 = Transaction("test_1", [
    Operation("append value", test_target, test_function),
    Operation("append second", test_target, test_function),
])

test_transa_2 = Transaction("test_2", [
    Operation("times 2", test_target, lambda x: x*2)
])

run_transactions(100, test_transa_1, test_transa_2)

test_1 Get lock of current operation append value
test_1 Apply current operation append value
test_1 Free lock of current operation append value
test_1 Get lock of current operation append second
test_1 Apply current operation append second
test_2 Get lock of current operation times 2
test_2 Get lock of current operation times 2
test_1 Free lock of current operation append second
test_2 Get lock of current operation times 2
test_2 Apply current operation times 2
test_2 Free lock of current operation times 2


In [63]:
test_target.df

,test
0,2
1,4
2,6
3,110
4,10
5,10
